<a href="https://colab.research.google.com/github/MaximNetolog/index.html-/blob/main/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%BE%D1%82%D1%87%D1%91%D1%82_%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%BD%D0%B5%D1%82_%D0%BC%D0%B0%D0%B3%D0%B0%D0%B7%D0%B8%D0%BD%D0%B0_%D0%B8%D0%B7_%D0%92%D0%B5%D0%BB%D0%B8%D0%BA%D0%BE%D0%B1%D1%80%D0%B8%D1%82%D0%B0%D0%BD%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Аналитический отчёт базы транзакций интернет-магазина из Великобритании.

Данные для анализа предоставлены по ссылке https://u.netology.ru/backend/uploads/lms/content_assets/file/3711/Data_Sample_AIC_new.xlsx преподавателем Арсений Совапо.

Для изучения данных использоваться язык программирования Python в стеде программирования Google Colab, который можно посмотреть по ссылке https://colab.research.google.com/drive/1XXbwZuFTNbO-3qxGYrJs7ht35pRM0PsR?usp=sharing#scrollTo=4wjKDQBMcWZs

# Описание данных:
Предоставленные данные находятся в файле, объёмом 3.10 МБ с названием "Data_Sample_AIC_new1", в формате xlsx.
Файл содержат 8 колонок. Каждая колонка имеет заголовок:

 - InvoiceNo - номер счёта-фактуры
 - StockCode - код товара
 - Description - описание товара
 - Quantity - количество товара
 - InvoiceDate - дата и время создания счёта
 - UnitPrice - цена за единицу товара
 - CustomerID - идентификатор покупателя
 - Country - страна

Для удобства исследования, колонки переименованы: "счет_фактура", "код_товара", "описание", "количество", "datetime_счёта", "цена_за_единицу", "id_клиента", "страна"

Согласно, табличной форме, все колонки должны иметь по 54214, но фактически, колонка 'описание' имеет 134 пропуска, а 'id_клиента' 13571 пропуска.

Колонки: "счет_фактура", "код_товара", "описание" и "страна" имеют тип данных - object.

Колонка: "количество" с типом - int64.

Колонка: "datetime_счёта" с типом - datetime64[ns]

Колонки: "цена_за_единицу" и "id_клиента" с типом - float64


# Справочная информация

При анализе предоставленных данных, необходимо учитывать, что все цены указаны в национальной валюте Великобритании фунт стерлингов (GBP).


# Описание применённого подхода к анализу данных


В целом, типы данных корректн, но при полном заполнении, колонка 'id_клиента' могла бы иметь тип int32. После заполнения пропусков в колонке 'id_клиента' тип данных был заменен на int32.

Колонка "счет_фактура" имеет тип object, в связи с корректирующими счетами-фактуры, создаваемыми при возврате товара клиентом в магазин. Нумерация таких счетов-фактуры формируется на основании номера исходного счёта-фактуры по которому проходила продажа. К этому номеру добавляется префикс "C", обозначающий "Credit" (кредитовую нота). Это документ, который выдается покупателю для учета возврата товара. Если в графе "количество товара" указано отрицательное число, это указывает на то, что товар был возвращен, и счет-фактура с префиксом "C" служит подтверждением возврата или предстоящего возврата средств или зачета суммы в счет будущих покупок.

Колонка Столбец "количество" содержит не только положительные, но и отрицательные значения. Из 1083 записей:

960 отрицательных записей относятся к корректирующим счетам-фактуры.

123 отрицательных записей относятся к счетам списаний

Колонка "описание" имеет пустые значения только для счетов-фактуры с продажами (номер счёта-фактуры без префикса "С"). Вероятно, система не даёт создать счёт-фактуру на возврат без заполнения поля "описание". Все пропуски в колонке "описане" соотносятся с нулевым значением цены и только к Великобритании. Таким образом, если выбрать любой счет-фактуру с пропуском в колонке "описание", то в колонке "цена" будет 0, а в колонке "страна" United Kingdom. Этот факт выявляет серьёзную уязвимость бухгалтерской системы учёта.

Колонка "цена_за_единицу" содержит нулевые значения в счетах-фактуры созданных для списаний и подарков. Если счёт-фактура содержит положительное количество товаров, то это подарки клиентам (цена за товар = 0). Если счёт-фактура содержит отрицательное количество товаров и при этом цена = 0, то это списание товара. При этом номер счёта-фактуры не содержит префикса "С". Всего за год сделано 134 подарка и 131 списание товаров. Необходимо учесть, что количество списанных товаров, так и количество подаренных товаров в каждом счёте может быть несколько. Т.е. количество счетов со списаниями и подарками всегда меньше или равно количеству товаров в них списанных и подаренных.

Для более полного отражения в движении денежных средств, нами создана колонка "сумма_по_счёту". Она рассчитывается как перемножение количества товара на
стоимость единицы товара.

Серьёзный недостаток предоставленных данных в отсутствии информации по продажам в других интернет-магазинах исследуемой торговой сети. Можно только надеяться, что при возврате, какм-то образом проверяются номера счетов-фактуры, код товара, количество, дата продажи, цена и id_клиента. В противном случае существует пространство для мошеннических схем как со стороны покупателей, так и со стороны работников магазина.

Так-же вызывает тревогу оформление списаний товара и оформление подарков. Отсутствие причины списания в колонке "описание", приводит к дополнительным вопросам.

# Описание бизнес-заказчика, его гипотетических болей и целей развития бизнеса.

Владелец интернет-магазина часто берет на себя функции заказчика бизнеса. Он может столкнуться с такими задачами по решению проблем: по сокращению числа возвратов товаров, мошенничества со стороны клиентов и работников магазина, снижению потерь клиентов, уменьшению случаев когда покупки не завершаются (товары остаются в корзине). Решение проблем развития бизнеса, увеличению общего числа продаж, повышениюе общей прибыли магазина.

# Рекомендации по модели данных бизнес-заказчика:

Для наглядности, дадим рекомендации на примере возможных скрытых убытков:

1. Возврат по счёту C545847 сделан 2020-03-07. В рассматриваемом магазине счёта 545847 нет, а по законадательству Великобритании, возврат товара осуществляется не позднее 30 суток с момента покупки. Это означает, что продажу сделал другой интернет магазин нашей торговой компании.
  - Рекомендация: предоставить данные по продажам в других магазинах для проверки всех подобных случаев.

2. Выявлено, что максимальная сумма возврата больше максимальной суммы покупки.Счет-фактура с номером 551699 только одна: C551699 - это возврат. Дата её создания 2020-05-03, Manual в количестве 1 шт, цена 6930. Это означает, что покупка сделана в другом магазине нашей сети продаж, а возврат проведён в проверяемом магазине. Самый большой вопрос к цене мануала, по которому сделан возврат, она больше средней стоимости автомобиля!
   - Рекомендация: Требуется проверка не только аномальных списаний, но для этого необходимы данные по всей сети магазинов

3. Самые частые изменения цен имеют различные сборы, комиссии, проценты, дисконты, скидки, почтовые расходы: AMAZONFEE, Manual, DOTCOM POSTAGE, CRUK Commission, BANK CHARGES. В отличии от них, мануалы (код товара "M" и описание "Manual") может являтся товаром, но у него так же большое количество разнообразных цен.
   - Рекомендация: Требуется точная фиксация ценообразования

4. В группе товаров с кодом "М" и описанием "Manual" нет ни одиной пары для счёта-фактуры возврата! Т.е. все возвраты мануалов с других интернет магазинов.
   - Рекомендация: требуется масштабировать проверку возвратов по всем группам товаров.

5. Самое большое списание № 556691 с кодом товара 23005 "КОШЕЛЕК С ПРОЕЗДНЫМИ КАРТАМИ "Я ЛЮБЛЮ ЛОНДОН" проведено 2020-06-14 в середине года, на сумму 5114.48 по причине:"пятна от печати/выброшенные".
  - Рекомендация: желательно проверять аномальные списания

6. По товару 79030G "TUMBLER, NEW ENGLAND" наблюдаем стабильные продажи в течении 5-и месяцев и не одного возврата! Вдруг списание по счёту 554143, с невнятным комментарием "damages". Такой комментарий не обьясняет причину "убытка".
  - Рекомендация: Необходима проверка всех фактов списания!
